In [303]:
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

In [421]:
# 필요데이터만 뽑아서 df불러오기
def read_data(year,month):
    if month < 10:
        month = '0' + str(month)
    data_name = 'data/' + str(year) + str(month) + '.csv'
    df = pd.read_csv(data_name, encoding='cp949', header=None)
    df = df.iloc[:, [0,2,5,6]]
    weight_name = str(year) + '-' + str(month) + '중량'
    price_name = str(year) + '-' + str(month) + '금액'
    df.columns = ['AGCODE','품목', weight_name, price_name]
    
    # 2016~2019년 데이터에서 품목컬럼에서 NAN값 존재하여 다음 코드 실행에 오류발생
    # -> 확인결과 AGCODE : 169220450 값으로 제거대상이므로 삭제
    df.dropna(inplace=True)

    # 농산물 데이터만 가져오기 : ASCODE -> 1로 시작
    df.AGCODE = df.AGCODE.astype(str)
    df = df[df.AGCODE.str.startswith('1')]
    
        
    # 중분류 데이터 가져오기
    df = df[df.AGCODE.str.len()==3]


    # 필요없는 데이터 제거
    drop_items = ['114', '119', '131', '132', '134', '137', '139', '141', '142', '144', '151', '152', '153', '155', '156',
                 '158', '159', '161', '169', '170', '171', '174']
    for item in drop_items:
        df = df.drop(df[df.AGCODE.str.startswith(item)].index)

    # 중량, 금액 타입 변환
    df[weight_name] = df[weight_name].str.replace(',','')
    df[price_name] = df[price_name].str.replace(',','')
    df[[weight_name, price_name]] = df[[weight_name, price_name]].apply(pd.to_numeric)
    
    return df

In [405]:
# 년도별로 df 병합시켜주는 함수
def merge_data(year):
    df = read_data(year,1)
    for i in range(2,13):
        df2 = read_data(year,i)
        df = pd.merge(df, df2, how = 'outer')
    return df

In [432]:
# 품목 이름 정리 ( -계 제거 ) 함수
def substr(df):
    df['품목'] = df['품목'].str[:-2]
    return df

### 코드
- 111 : 곡류  
- 112 : 서류  
- 113 : 두류  
- 115 : 채유종실  
- 116 : 과실류  
- 121 : 채소류  
- 123 : 버섯류  
- 143 : 소스류  
- 157 : 주류  
- 162 : 면류  

In [429]:
# 2016 ~ 2020년 df 생성
df_2020 = substr(merge_data(2020))
df_2019 = substr(merge_data(2019))
df_2018 = substr(merge_data(2018))
df_2017 = substr(merge_data(2017))
df_2016 = substr(merge_data(2016))

In [439]:
df_2020.to_csv('data/농산물_df_2020.csv', index = False)
df_2019.to_csv('data/농산물_df_2019.csv', index = False)
df_2018.to_csv('data/농산물_df_2018.csv', index = False)
df_2017.to_csv('data/농산물_df_2017.csv', index = False)
df_2016.to_csv('data/농산물_df_2016.csv', index = False)

In [445]:
df_2020

,AGCODE,품목,2020-01중량,2020-01금액,2020-02중량,2020-02금액,2020-03중량,2020-03금액,2020-04중량,2020-04금액,...,2020-08중량,2020-08금액,2020-09중량,2020-09금액,2020-10중량,2020-10금액,2020-11중량,2020-11금액,2020-12중량,2020-12금액
0,111,곡류,4011371.9,2517718.0,5863216.7,5058670.0,4183218.1,3417004.0,41278279.0,15679101.0,...,4584898.9,4310016.0,4965122.1,4781260.0,3623797.6,2673341.0,4598579.1,3797940.0,4294031.9,3198445.0
1,112,서류,45885.6,221049.0,130029.9,506828.0,128478.7,529760.0,100839.7,572611.0,...,78298.7,526421.0,117381.3,574239.0,158376.2,719438.0,113415.6,703681.0,129428.3,977026.0
2,113,두류,132517.3,227677.0,132783.5,260947.0,178046.1,330598.0,116752.8,163841.0,...,83655.4,196802.0,136333.4,279878.0,127142.1,278907.0,124691.4,263643.0,114181.7,305130.0
3,115,채유종실,3672.7,53705.0,6880.6,71831.0,9647.5,69444.0,31015.6,110739.0,...,7694.1,61988.0,25585.4,110186.0,9804.8,77035.0,13493.5,60895.0,3545.7,41313.0
4,116,과실류,9706711.9,29915537.0,8160085.2,22815037.0,8886397.1,21440028.0,6193232.9,15073846.0,...,9554385.7,25160321.0,14632351.0,42427783.0,13687998.1,40211748.0,19634399.9,52915536.0,16408194.3,46722844.0
5,121,채소류,7911968.1,32461658.0,10805153.6,35271969.0,10475343.0,37429506.0,12602338.5,40152105.0,...,8764070.8,28814092.0,5734933.6,22270808.0,7679806.6,23988776.0,16182955.5,36451633.0,14368831.5,42865086.0
6,123,버섯류(농산물),2088336.6,5179764.0,2088310.6,5111907.0,1424948.1,3657578.0,905931.5,2381776.0,...,1525121.3,3783506.0,1610126.5,4168713.0,1416777.5,3782934.0,1605326.3,4424837.0,1864233.7,5326937.0
7,143,소스류,7705737.4,20282760.0,8297117.1,21869523.0,9840253.2,26711027.0,9319525.0,26041838.0,...,9895722.2,25209186.0,11445981.5,30454492.0,10800258.0,28400977.0,11018359.9,28733506.0,11305438.6,30530248.0
8,145,지방성물질,1719940.0,255622.0,2419210.0,409301.0,3417092.5,565651.0,2693050.0,499183.0,...,2656190.0,509595.0,3118801.0,717682.0,2454591.0,625831.0,1763690.0,476245.0,1761423.5,576379.0
9,157,주류,23002674.0,21666329.0,27080924.3,24696363.0,28614386.7,26676408.0,28179756.0,27561734.0,...,26638893.9,23416088.0,29990826.4,29272955.0,25469931.2,25842389.0,31786723.8,31435759.0,24190937.2,26488958.0
